In [1]:
stones = [[] for _ in range(100)]
blocks = [[] for _ in range(100)]

with open("./data/day14.txt") as f:
    row = 0
    while line := f.readline():
        line = line.rstrip()
        for col, c in enumerate(line):
            if c == 'O':
                stones[col].append(row)
            elif c == '#':
                blocks[col].append(row)
        row += 1

In [2]:
stones[0], blocks[0]

([7, 18, 19, 25, 34, 46, 47, 49, 52, 57, 74, 85, 94, 98],
 [14, 21, 26, 29, 31, 51, 56, 58, 64, 66, 71, 72, 76, 80, 81, 84, 92, 96])

In [3]:
from bisect import bisect_left

In [4]:
for i in range(len(stones)):
    for stone_ind in range(len(stones[i])):
        row = stones[i][stone_ind]
        new_row = 0
        above_block = bisect_left(blocks[i], row)
        if above_block > 0:
            new_row = max(new_row, blocks[i][above_block-1]+1)
        if stone_ind > 0:
            new_row = max(new_row, stones[i][stone_ind-1]+1)
        stones[i][stone_ind] = new_row

In [5]:
stones[0], blocks[0]

([0, 15, 16, 22, 32, 33, 34, 35, 52, 57, 73, 85, 93, 97],
 [14, 21, 26, 29, 31, 51, 56, 58, 64, 66, 71, 72, 76, 80, 81, 84, 92, 96])

In [6]:
s = 0
for i in range(len(stones)):
    for row in stones[i]:
        s += 100-row
s

108955

Okay part 2 needs a lot of tilting so let's write some helper functions.

In [7]:
def tilt():
    for i in range(len(stones)):
        for stone_ind in range(len(stones[i])):
            row = stones[i][stone_ind]
            new_row = 0
            above_block = bisect_left(blocks[i], row)
            if above_block > 0:
                new_row = max(new_row, blocks[i][above_block-1]+1)
            if stone_ind > 0:
                new_row = max(new_row, stones[i][stone_ind-1]+1)
            stones[i][stone_ind] = new_row

In [8]:
def turn():
    global stones, blocks
    new_stones = [[] for _ in range(100)]
    new_blocks = [[] for _ in range(100)]
    for i in range(len(stones)):
        for j in range(len(stones[i])):
            new_stones[len(new_stones)-1-stones[i][j]].append(i)
    for i in range(len(blocks)):
        for j in range(len(blocks[i])):
            new_blocks[len(new_blocks)-1-blocks[i][j]].append(i)
    for i in range(len(new_stones)):
        new_stones[i].sort()
    for i in range(len(new_blocks)):
        new_blocks[i].sort()
    stones = new_stones
    blocks = new_blocks

There is no way we actually need to do all the cycles. The state of the rocks will certainly start looping. Let's find the loop. We need a helper function to save the current state to a dictionary.

In [9]:
def add_curr_state(ind):
    global states
    state = tuple()
    for i in range(len(stones)):
        for j in range(len(stones[i])):
            state += (i, stones[i][j])
    if state in states:
        return states[state]
    states[state] = ind
    return -3

In [10]:
states = {}
add_curr_state(0)

for i in range(1000):
    tilt()
    turn()
    tilt()
    turn()
    tilt()
    turn()
    tilt()
    turn()
    res = add_curr_state(i+1)
    if res != -3:
        print('found loop', res, i+1)
        break

found loop 85 107


Loop going from 85th state to 105th state. We need to find 1000000000th state to calculate answer.

1000000000 = 85 + n*(107-85) + x

x = (1000000000 - 85) % (107-85)

In [11]:
x = (1000000000-85)%(107-85)
x

13

In [12]:
ans_state = None
for k, v in states.items():
    if v == 85+x:
        ans_state = k
        break

In [13]:
stones = [[] for _ in range(100)]

for i in range(0, len(ans_state), 2):
    stones[ans_state[i]].append(ans_state[i+1])

In [14]:
s = 0
for i in range(len(stones)):
    for row in stones[i]:
        s += 100-row
s

106689